# Traitement des données récupérée sur le daemon
>scp zprojet@recupvol.istic.univ-rennes1.fr:~/deproject/allData.csv .


### Installs

In [1]:
!pip install pandas
!pip install seaborn
!pip install matplotlib
!pip install impyute
!pip install sklearn
!pip install metar-taf-parser-mivek

In [5]:
import pandas as pd
import seaborn as sb
import numpy as np
from metar_taf_parser.parser.parser import MetarParser
import matplotlib.pyplot as plt
from impyute.imputation.cs import mice
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import brier_score_loss
from sklearn.metrics import f1_score


## Machine Learning


In [17]:
dfAll=pd.read_csv("allDataMieux.csv")
dfAll=dfAll.dropna()
dfAll.head(1)

,Unnamed: 0,METAR,DateTime UTC,Time,Flight,To,Airline,Aircraft,Status,FullDate,secRetard,iata
0,0.0,DTTA 271500Z 28005KT 230V320 9999 SCT026 18/04...,2021-11-27 15:00:00,14:55,EK748,Dubai (DXB),Emirates,B77W (A6-EGX),Departed 15:01,2021-11-27 15:00:00,360.0,tun


## interprétation code METAR

In [ ]:
#creation nouvelle colonnes
dfAll["WindSpeed"]=""
dfAll["WindDirection"]=""
dfAll["Temperature"]=""
dfAll["DewPoint"]=""
dfAll["Visibility"]=""
dfAll["Altimeter"]=""

  #ajout de toutes les valeurs dans les colonnes ajoutées

for index, row in dfAll.iterrows() :
  try:
    metar = MetarParser().parse(str(row["METAR"]))
    row["WindSpeed"]=metar.wind.speed
    row["WindDirection"]=metar.wind.degrees
    row["Temperature"]=metar.temperature
    row["DewPoint"]=metar.dew_point
    row["Visibility"]=metar.visibility.distance
    row["Altimeter"]=metar.altimeter
    dfAll.iloc[index]=row
  except:
    continue

dfAll


,Unnamed: 0,METAR,DateTime UTC,Time,Flight,To,Airline,Aircraft,Status,FullDate,secRetard,iata,WindSpeed,WindDirection,Temperature,DewPoint,Visibility,Altimeter
0,0.0,DTTA 271500Z 28005KT 230V320 9999 SCT026 18/04...,2021-11-27 15:00:00,14:55,EK748,Dubai (DXB),Emirates,B77W (A6-EGX),Departed 15:01,2021-11-27 15:00:00,360.0,tun,5,280,18,4,> 10km,1004
1,1.0,DTTA 271500Z 28005KT 230V320 9999 SCT026 18/04...,2021-11-27 15:00:00,15:00,TU850,Lyon (LYS),Tunisair,A320 (TS-IMV),Departed 15:22,2021-11-27 15:00:00,1320.0,tun,5,280,18,4,> 10km,1004
2,2.0,DTTA 271500Z 28005KT 230V320 9999 SCT026 18/04...,2021-11-27 15:00:00,15:00,UG8,Djerba (DJE),Tunisair Express,AT76 (TS-LBF),Departed 15:12,2021-11-27 15:00:00,720.0,tun,5,280,18,4,> 10km,1004
3,3.0,DTTA 271430Z 24010KT 200V270 9999 SCT026 18/04...,2021-11-27 14:30:00,13:50,TU790,London (LHR),Tunisair,A320 (TS-IMR),Departed 14:50,2021-11-27 14:30:00,3600.0,tun,10,240,18,4,> 10km,1004
4,4.0,DTTA 271330Z 26009KT 220V280 9999 SCT026 18/04...,2021-11-27 13:30:00,12:55,TU902,Marseille (MRS),Tunisair,A320 (TS-IMU),Departed 13:55,2021-11-27 13:30:00,3600.0,tun,9,260,18,4,> 10km,1004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84310,5.0,EFHK 192250Z 36015KT 9999 -SN BKN026 M05/M11 Q...,2021-12-19 22:30:00,22:30,AY961,Copenhagen (CPH),Finnair,E190 (OH-LKL),Departed 22:33,2021-12-19 22:30:00,180.0,hel,15,360,-5,-11,> 10km,1010
84311,6.0,EFHK 192250Z 36015KT 9999 -SN BKN026 M05/M11 Q...,2021-12-19 22:30:00,22:30,OJ429,Kokkola (KOK),NyxAir,SB20 (ES-NSH),Departed 22:40,2021-12-19 22:30:00,600.0,hel,15,360,-5,-11,> 10km,1010
84312,7.0,EFHK 192220Z 01014KT 9999 -SN BKN027 M04/M10 Q...,2021-12-19 22:00:00,21:20,AY537,Rovaniemi (RVN),Finnair (Oneworld Livery),A319 (OH-LVD),Departed 22:28,2021-12-19 22:00:00,4080.0,hel,14,10,-4,-10,> 10km,1009
84313,8.0,EFHK 192220Z 01014KT 9999 -SN BKN027 M04/M10 Q...,2021-12-19 22:00:00,22:00,SK719,Stockholm (ARN),Xfly,CRJ9 (ES-ACP),Departed 22:22,2021-12-19 22:00:00,1320.0,hel,14,10,-4,-10,> 10km,1009


### optional code

In [9]:
dfAll=pd.read_csv("AllData.csv")

### changing values to numeric ones

In [10]:
dfAll=dfAll.dropna()
dfAll = dfAll[dfAll['WindSpeed']!="WindSpeed"]

In [11]:
GRElabels = dfAll["Airline"].unique().tolist()
mapping = dict(zip(GRElabels,range(len(GRElabels))))
dfAll.replace({"Airline":mapping},inplace=True)

GRElabels = dfAll["iata"].unique().tolist()
mapping = dict(zip(GRElabels,range(len(GRElabels))))
dfAll.replace({"iata":mapping},inplace=True)
dfAll["Visibility"] = dfAll["Visibility"].replace({"> 10km":"10000"})


In [12]:
#dropping useless data
dfAll=dfAll.drop(columns=["METAR","DateTime UTC","Time","Flight","To","Aircraft","Status","FullDate"]) #.astype('int32')
dfAll=dfAll.drop(columns=dfAll.columns[0])
dfAll=dfAll.drop(columns=dfAll.columns[0])
dfAll

,Airline,secRetard,iata,WindSpeed,WindDirection,Temperature,DewPoint,Visibility,Altimeter
0,0,360.0,0,5.0,280.0,18.0,4.0,10000,1004.0
1,1,1320.0,0,5.0,280.0,18.0,4.0,10000,1004.0
2,2,720.0,0,5.0,280.0,18.0,4.0,10000,1004.0
3,1,3600.0,0,10.0,240.0,18.0,4.0,10000,1004.0
4,1,3600.0,0,9.0,260.0,18.0,4.0,10000,1004.0
...,...,...,...,...,...,...,...,...,...
77071,31,180.0,18,15.0,360.0,-5.0,-11.0,10000,1010.0
77072,505,600.0,18,15.0,360.0,-5.0,-11.0,10000,1010.0
77073,508,4080.0,18,14.0,10.0,-4.0,-10.0,10000,1009.0
77074,235,1320.0,18,14.0,10.0,-4.0,-10.0,10000,1009.0


### Type conversion

In [13]:
dfAll["Airline"]=pd.to_numeric(dfAll["Airline"],errors='coerce').convert_dtypes()
dfAll["WindSpeed"]=pd.to_numeric(dfAll["WindSpeed"],errors='coerce').convert_dtypes()
dfAll["WindDirection"]=pd.to_numeric(dfAll["WindDirection"],errors='coerce').convert_dtypes()
dfAll["Temperature"]=pd.to_numeric(dfAll["Temperature"],errors='coerce').convert_dtypes()
dfAll["DewPoint"]=pd.to_numeric(dfAll["DewPoint"],errors='coerce').convert_dtypes()
dfAll["Visibility"]=pd.to_numeric(dfAll["Visibility"],errors='coerce').convert_dtypes()
dfAll["Altimeter"]=pd.to_numeric(dfAll["Altimeter"],errors='coerce').convert_dtypes()
dfAll["secRetard"]=pd.to_numeric(dfAll["secRetard"],errors='coerce').convert_dtypes()
dfAll = dfAll.dropna()

## model Building and evaluation

### Création des vecteurs

In [14]:
X = dfAll.drop(["secRetard"],axis=1)
Y = dfAll["secRetard"]
tabScores=[]
X=X.astype(int)
Y=Y.astype(int)

### Logistic Regression

In [9]:
LgReg = LogisticRegression(max_iter=700)#obligé de changer le nombre max d'iterations sinon insuffisant
LgReg.fit(X,Y)
tabScores.append(LgReg.score(X,Y))

D:\Program Files (x86)\Python9\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Support Vector Machine

In [15]:
supp = svm.SVC()
supp.fit(X, Y)
tabScores.append(supp.score(X,Y))

###Gaussian Naive Bayes

In [25]:
gaussian = GaussianNB()
gaussian.fit(X, Y)
tabScores.append(gaussian.score(X,Y))

###Decision Tree

In [26]:
dectree = tree.DecisionTreeClassifier()
dectree.fit(X, Y)
tabScores.append(dectree.score(X,Y))

### Random Forest

In [27]:
rdf = RandomForestClassifier()
rdf.fit(X, Y)
tabScores.append(rdf.score(X,Y))

### scores(non représentatif)

In [28]:
print(tabScores)

[0.03641281731169372, 0.035134653112181204, 0.8174305927114915, 0.8174305927114915]


## K-fold cross validation

In [29]:
tabScoresCrossVal = []
tabScoresCrossVal.append(cross_val_score(LgReg, X, Y, cv=10).mean())
tabScoresCrossVal.append(cross_val_score(supp, X, Y, cv=10).mean())
tabScoresCrossVal.append(cross_val_score(gaussian, X, Y, cv=10).mean())
tabScoresCrossVal.append(cross_val_score(dectree, X, Y, cv=10).mean())
tabScoresCrossVal.append(cross_val_score(rdf, X, Y, cv=10).mean())
tabScoresCrossVal

D:\Program Files (x86)\Python9\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"
D:\Program Files (x86)\Python9\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
D:\Program Files (x86)\Python9\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter

D:\Program Files (x86)\Python9\lib\site-packages\sklearn\model_selection\_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "D:\Program Files (x86)\Python9\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "D:\Program Files (x86)\Python9\lib\site-packages\sklearn\ensemble\_forest.py", line 387, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "D:\Program Files (x86)\Python9\lib\site-packages\joblib\parallel.py", line 1044, in __call__
    while self.dispatch_one_batch(iterator):
  File "D:\Program Files (x86)\Python9\lib\site-packages\joblib\parallel.py", line 859, in dispatch_one_batch
    self._dispatch(tasks)
  File "D:\Program Files (x86)\Python9\lib\site-packages\joblib\parallel.py", line 777, in _dispatch
    job = self._

D:\Program Files (x86)\Python9\lib\site-packages\sklearn\model_selection\_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "D:\Program Files (x86)\Python9\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "D:\Program Files (x86)\Python9\lib\site-packages\sklearn\ensemble\_forest.py", line 387, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "D:\Program Files (x86)\Python9\lib\site-packages\joblib\parallel.py", line 1044, in __call__
    while self.dispatch_one_batch(iterator):
  File "D:\Program Files (x86)\Python9\lib\site-packages\joblib\parallel.py", line 859, in dispatch_one_batch
    self._dispatch(tasks)
  File "D:\Program Files (x86)\Python9\lib\site-packages\joblib\parallel.py", line 777, in _dispatch
    job = self._

D:\Program Files (x86)\Python9\lib\site-packages\sklearn\model_selection\_validation.py:615: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "D:\Program Files (x86)\Python9\lib\site-packages\sklearn\model_selection\_validation.py", line 598, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "D:\Program Files (x86)\Python9\lib\site-packages\sklearn\ensemble\_forest.py", line 387, in fit
    trees = Parallel(n_jobs=self.n_jobs, verbose=self.verbose,
  File "D:\Program Files (x86)\Python9\lib\site-packages\joblib\parallel.py", line 1044, in __call__
    while self.dispatch_one_batch(iterator):
  File "D:\Program Files (x86)\Python9\lib\site-packages\joblib\parallel.py", line 859, in dispatch_one_batch
    self._dispatch(tasks)
  File "D:\Program Files (x86)\Python9\lib\site-packages\joblib\parallel.py", line 777, in _dispatch
    job = self._

[0.0324000296823459,
 0.03641280899233736,
 0.021491281694296573,
 0.03674108160348273,
 nan]

### Test sur des données
-On utilise le "SVM" classifier pour tester sur des données rentrées à la main

In [28]:
test = pd.DataFrame({"Airline":4.0,"iata":0,"WindSpeed":15.0,"WindDirection":280.0,
                     "Temperature":5.0,"DewPoint":4.0,"Visibility":100.0,"Altimeter":1004.0},index=[1])

In [30]:
res = supp.predict(test)
res

array([420])